Latent Dirichlet Allocation - Variational Inference
====

Based on the paper "Latent Dirchlet Allocation" by David M. Blei, Andrew Y. Ng, Michael I. Jordan

In [1]:
import numpy as np
from numpy import sqrt,mean,square
import numpy.linalg as la
from scipy.special import digamma, polygamma

In [2]:
!git config --global user.email "kevinjliang2011@gmail.com"
!git config --global user.name "Kevin Liang"

## Parameters

document:    $m = 1,...,M$

topic:       $z = 1,...,k$

word:        $w = 1,...,N_m$

vocabulary : $v = 1,...,V$

$\alpha: 1 \times k$ vector of topic distribution probabilities

$\beta: k \times v$ matrix of word probabilities for each topic

$\phi: M \times N_m \times k$ matrix of topic probabilities for each word in each document

$\gamma: M \times k$ matrix of topic probabilities for each document

In [3]:
np.random.seed(1337)

### Test data and pre-processing

Run the following first:
```
pip install -U nltk
pip install stop-words
pip install -U gensim
```

In [4]:
!pip install -U nltk
!pip install stop-words
!conda install -y gensim

Requirement already up-to-date: nltk in /Users/andrea/anaconda3/lib/python3.5/site-packages
Fetching package metadata: ....
Solving package specifications: .........

Package plan for installation in environment /Users/andrea/anaconda3:

The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bz2file-0.98               |           py35_0          10 KB
    httpretty-0.8.14           |           py35_0          31 KB
    smart_open-1.3.2           |           py35_0          28 KB
    gensim-0.12.4              |      np110py35_0         2.3 MB
    ------------------------------------------------------------
                                           Total:         2.3 MB

The following NEW packages will be INSTALLED:

    bz2file:    0.98-py35_0       
    gensim:     0.12.4-np110py35_0
    httpretty:  0.8.14-py35_0     
    smart_open: 1.3.2-py35_0      

Fetching packages ...
bz2file-0.98-p 10

In [5]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health." 

# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [6]:
corpus

[[(0, 2), (1, 2), (2, 2), (3, 1), (4, 1), (5, 1)],
 [(3, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)],
 [(6, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1)],
 [(3, 1),
  (5, 1),
  (6, 1),
  (14, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1)],
 [(1, 1), (2, 1), (13, 2), (30, 1), (31, 1)]]

### Randomly generated test data

In [8]:
M = 200
k = 10
N = np.random.randint(50,100,size=M)
V = 100

print('N: {0}'.format(N))

N: [91 84 89 81 94 72 71 88 51 64 88 80 51 77 87 97 50 64 67 52 82 86 63 91 55
 51 83 97 73 65 87 54 54 72 81 98 94 81 80 92 93 55 59 84 51 93 74 78 98 98
 92 59 56 94 86 63 57 57 75 71 57 83 91 87 75 61 68 59 57 74 79 95 77 93 94
 94 84 64 73 68 89 72 64 79 60 68 78 82 76 82 77 82 85 72 65 95 89 99 85 54
 73 59 86 55 74 85 97 75 78 85 66 81 51 65 59 69 81 85 52 78 92 51 65 90 68
 74 87 53 71 82 86 69 66 62 58 98 70 99 66 89 92 73 63 76 92 50 82 57 78 61
 54 85 86 74 87 94 67 70 62 69 81 63 53 68 97 89 51 71 60 68 83 95 99 54 75
 98 83 90 65 54 98 82 82 99 58 60 88 60 90 93 51 59 86 53 93 54 85 65 76 89]


#### Completely random structure

In [23]:
# Generate random "documents"
doc1 = np.random.randint(V,size=N[0])
doc2 = np.random.randint(V,size=N[1])
doc3 = np.random.randint(V,size=N[2])

w = np.array((doc1,doc2,doc3))

#### Add some structure

In [15]:
alpha_gen = np.array((1,1,10,1,1,7,1,20,1,1))
beta_gen = np.random.dirichlet(0.1*np.ones(V),k).T

w_struct = list();

for m in range(M):
    theta = np.random.dirichlet(alpha_gen,1)[0]
    doc = np.array([])
    for n in range(N[m]):
        z_n = np.random.choice(np.arange(k),p=theta)
        w_n = np.random.choice(np.arange(V),p=beta_gen[:,z_n])
        doc = np.append(doc,w_n)
    w_struct.append(doc)

In [21]:
w_struct[1].shape

(84,)

### Initialize parameters $\alpha, \beta, \phi$ and $\gamma$

In [36]:
alpha = np.random.dirichlet(np.ones(k),1)[0]
beta = np.random.dirichlet(np.ones(V),k).T

phi = np.array([1/k*np.ones([N[m],k]) for m in range(M)])
gamma = np.tile(alpha,(M,1)) + np.tile(N/k,(k,1)).T

In [37]:
alpha

array([ 0.19931287,  0.09150602,  0.01462898,  0.09046449,  0.15424888,
        0.04798922,  0.05593922,  0.03454043,  0.16268027,  0.14868963])

In [36]:
beta.shape

(20, 10)

In [42]:
phi.shape

(3,)

In [39]:
gamma.shape

(3, 10)

### Optimize variational parameters $\phi$ and $\gamma$

In [10]:
# TODO: Split phi and gamma optimization apart for parallelization purposes
# TODO: See if some sort of vectorization is possible for speed-up
def optVarParams(alpha,beta,phi,gamma,words):
    ## Optimize phi
    for m in range(M):
        for n in range(N[m]):
            for i in range(k):
                phi[m][n,i] = beta[words[m][n],i] * np.exp(digamma(gamma[m,i]) - digamma(np.sum(gamma[m,:])))
            # Normalize across states so phi represents probability over states for each word
            phi[m][n,:] = phi[m][n,:]/sum(phi[m][n,:])
    
    ## Optimize gamma
    gamma = np.tile(alpha,(M,1)) + np.array(list(map(lambda x: np.sum(x,axis=0),phi)))
    
    return phi,gamma

In [96]:
phi,gamma = optVarParams(alpha,beta,phi,gamma,w_struct)

/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:8: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [97]:
phi

array([ array([[ 0.07243487,  0.11005002,  0.10910023,  0.07741408,  0.0930058 ,
         0.09731828,  0.08083785,  0.1848577 ,  0.08452286,  0.09045831],
       [ 0.07064096,  0.110401  ,  0.10938119,  0.07582152,  0.09221637,
         0.09679324,  0.07939991,  0.19255947,  0.08326543,  0.08952091],
       [ 0.07137439,  0.11025825,  0.10926707,  0.07647285,  0.0925398 ,
         0.09700864,  0.07998815,  0.18940608,  0.08378   ,  0.08990478],
       [ 0.07470869,  0.1096044 ,  0.10874335,  0.07943248,  0.09400574,
         0.09798304,  0.08266013,  0.17510009,  0.08611622,  0.09164586],
       [ 0.07470869,  0.1096044 ,  0.10874335,  0.07943248,  0.09400574,
         0.09798304,  0.08266013,  0.17510009,  0.08611622,  0.09164586],
       [ 0.07137439,  0.11025825,  0.10926707,  0.07647285,  0.0925398 ,
         0.09700864,  0.07998815,  0.18940608,  0.08378   ,  0.08990478],
       [ 0.07470869,  0.1096044 ,  0.10874335,  0.07943248,  0.09400574,
         0.09798304,  0.08266013,  0.

### Estimate model parameters $\alpha$ and $\beta$

In [43]:
def estModParams(alpha,beta,phi,gamma,words):
    ## Optimize beta
    for j in range (V):
        w_dnj = [np.tile((word == j),(k,1)).T for word in words]
        beta[j,:] = np.sum(np.array(list(map(lambda x: np.sum(x,axis=0),phi*w_dnj))),axis=0)
        
    # Normalize across states so beta represents probability of each word given the state
    for i in range(k):
        beta[:,i] = beta[:,i]/sum(beta[:,i])
    
    ## Optimize alpha
    nr_max_iters = 1000
    tol = 10**-3
    for it in range(nr_max_iters):
        alpha_old = alpha
        
        #  Calculate gradient 
        g = M*(digamma(np.sum(alpha))-digamma(alpha)) + np.sum(digamma(gamma)-np.tile(digamma(np.sum(gamma,axis=1)),(k,1)).T,axis=0)
        #  Calculate Hessian diagonal component
        h = -M*polygamma(1,alpha) 
        #  Calculate Hessian constant component
        z = polygamma(1,np.sum(alpha))
        #  Calculate constant
        c = np.sum(g/h)/(z**(-1)+np.sum(h**(-1)))

        #  Update alpha
        alpha = alpha - (g-c)/h
        
        #  Check convergence
        if sqrt(mean(square(alpha-alpha_old)))<tol:
            break

    return alpha,beta

In [98]:
estModParams(alpha,beta,phi,gamma,w_struct)

(array([ 2.25707885,  3.18005535,  3.15743801,  2.38338091,  2.76962137,
         2.87445828,  2.46930408,  4.90355497,  2.56103432,  2.7073288 ]),
 array([[ 0.0233352 ,  0.0215924 ,  0.02162572,  0.02304118,  0.02226148,
          0.02207672,  0.02285293,  0.01984268,  0.02266176,  0.02237629],
        [ 0.04272096,  0.04433923,  0.04430993,  0.04300445,  0.04373715,
          0.04390614,  0.04318402,  0.0457432 ,  0.04336471,  0.04363119],
        [ 0.03244107,  0.03309246,  0.03307874,  0.0325427 ,  0.03282741,
          0.03289851,  0.03260933,  0.03391304,  0.03267832,  0.03278398],
        [ 0.05354856,  0.05530391,  0.05526844,  0.05383252,  0.05460764,
          0.05479673,  0.05401657,  0.05732513,  0.0542054 ,  0.05449128],
        [ 0.04217531,  0.04439841,  0.04435275,  0.04253031,  0.04350796,
          0.04374855,  0.04276127,  0.04706261,  0.04299897,  0.04336032],
        [ 0.01187964,  0.01074244,  0.01076451,  0.01168992,  0.01118292,
          0.01106183,  0.01156805

### Expectation Maximization

#### Convergence Criterion
The variational inference parameter $\gamma$ contains the topic likelihoods of every document and is thus what is of interest here.

Calculate root-mean-square of the change in $\gamma$

In [12]:
def converged(gamma,gamma_old,convergence):
    print(sqrt(mean(square(gamma-gamma_old))))
    return sqrt(mean(square(gamma-gamma_old))) < convergence

#### Inference by iterative EM
Continue until convergence criterion above met

In [38]:
convergence = 10**(-2)
successfully_Converged = False
max_iters = 10**3

for iters in range(max_iters):
    print(iters)
    gamma_old = gamma
    phi,gamma  = optVarParams(alpha,beta,phi,gamma,w_struct)
    alpha,beta = estModParams(alpha,beta,phi,gamma,w_struct)
    if converged(gamma,gamma_old,convergence):
        successfully_Converged = True
        break

0


/opt/conda/lib/python3.4/site-packages/ipykernel/__main__.py:8: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


1.1728208738
1
6.71096430783
2
5.3404244674
3
5.25823606546
4
5.66772031452
5
6.02442485396
6
6.24899332366
7
6.38039989598
8
6.45942396586
9
6.5100442801
10
6.54446736808
11
6.56908430221
12
6.58720351389
13
6.59488886071
14
6.59425981897
15
6.58778768279
16
6.57595663829
17
6.55921123459
18
6.53798263659
19
6.51269763317
20
6.48377795713
21
6.45163565351
22
6.41666758656
23
6.37925056134
24
6.33973764257
25
6.29845578376
26
6.25570464971
27
6.2117564268
28
6.16685639081
29
6.12122402195
30
6.0750544857
31
6.02852033273
32
5.98177330723
33
5.9349461784
34
5.88815453714
35
5.84149851672
36
5.79506441254
37
5.74892618638
38
5.70314684761
39
5.65777971319
40
5.6128695458
41
5.56845357955
42
5.52456243795
43
5.48122095371
44
5.4384488971
45
5.39626162371
46
5.35467064581
47
5.31368413954
48
5.27330739027
49
5.23354318666
50
5.19439216606
51
5.15585311862
52
5.11792325474
53
5.08059843841
54
5.04387339347
55
5.007741882
56
4.97219686263
57
4.93723062681
58
4.90283491976
59
4.86900104365
60

In [39]:
alpha

array([ 2203.14755712,  1986.44953424,  2674.01606752,  2124.84049814,
        2348.52124005,  2093.71170047,  2295.74245208,  1690.72267503,
        2313.89598651,  1893.0053724 ])

In [40]:
alpha_gen

array([ 1,  1, 10,  1,  1,  7,  1, 20,  1,  1])

In [41]:
beta

array([[  1.01681267e-03,   2.80551061e-02,   5.87219151e-04,
          2.23607800e-03,   5.55162797e-04,   4.58406878e-03,
          2.54826459e-03,   2.12129784e-02,   7.72335334e-06,
          2.47214744e-02],
       [  1.63058960e-03,   1.99900179e-03,   2.14226224e-03,
          5.66530555e-03,   2.91492615e-03,   3.65421303e-04,
          3.60112583e-03,   4.36396536e-03,   9.11861265e-03,
          6.05571118e-03],
       [  1.06517874e-04,   3.30067331e-03,   2.28448751e-03,
          1.04066205e-03,   7.11286720e-03,   6.50555861e-04,
          2.95902639e-04,   3.13899377e-04,   2.05774199e-04,
          3.84212392e-04],
       [  4.86186835e-04,   3.31738867e-03,   1.60085894e-03,
          2.89826749e-03,   7.77348081e-04,   9.95726238e-03,
          8.05378730e-04,   6.04656966e-04,   6.73039184e-04,
          6.95643577e-03],
       [  1.59730280e-04,   1.09006287e-03,   4.19074676e-03,
          3.16309285e-04,   2.41010384e-04,   1.09445595e-03,
          1.35160723e-04

In [42]:
beta_gen

array([[  1.14172881e-04,   2.09085539e-02,   2.66549752e-02,
          8.68598267e-13,   1.49076140e-06,   5.36510562e-06,
          5.50858809e-05,   9.93421459e-04,   3.53465064e-04,
          5.28243981e-15],
       [  1.09436376e-02,   1.44149037e-04,   2.80988075e-04,
          1.74392995e-03,   1.67540004e-02,   5.15125622e-03,
          1.17313515e-04,   5.11573589e-04,   7.84783059e-02,
          3.45181042e-03],
       [  4.55411023e-05,   1.58096050e-07,   3.88079215e-03,
          5.89555869e-03,   2.27446128e-02,   5.26127402e-07,
          1.07982064e-02,   7.93285000e-05,   6.43837946e-04,
          2.43291375e-04],
       [  3.14623969e-02,   1.07611865e-03,   2.68801067e-26,
          1.81008712e-05,   2.82555446e-04,   1.67889777e-10,
          2.42845548e-02,   4.08520643e-03,   5.95698336e-06,
          3.70841977e-07],
       [  8.00215979e-08,   1.88495037e-10,   1.44717153e-03,
          3.57093062e-04,   3.00305465e-11,   9.31165503e-05,
          5.01579549e-06

### Tests 
Testing out syntax and array dimensions

In [11]:
# Word #11 in document 2 (w_dn)
w[1][10]

3

In [28]:
[doc == 3 for doc in w]

[array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False,  True, False, False], dtype=bool),
 array([False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False,
         True, False, False, False, False,  True, False, False, False, False], dtype=bool),
 array([False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False,  True,
        False, False, False, False], dtype=bool)]

In [29]:
[doc == 3 for doc in w]*w

array([ array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0]),
       array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0,
       3, 0, 0, 0, 0]),
       array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0])], dtype=object)

In [11]:
np.sum(np.array(list(map(lambda x: np.sum(x,axis=0),phi))),axis=0)

array([ 9.1,  9.1,  9.1,  9.1,  9.1,  9.1,  9.1,  9.1,  9.1,  9.1])